# Word2Vec 구현 프로젝트 

Efficient Estimation of Word Representations in Vector Space 

논문의 word2vec의 skip-gram 방식을 pytorch를 이용해 구현하고 학습하는 과정을 담았습니다. 

In [1]:
import os 
import sys
import pandas as pd
import numpy as np
import re
from typing import List, Dict
import random

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 61.7 MB/s 
     |████████████████████████████████| 6.6 MB 40.3 MB/s 
     |████████████████████████████████| 86 kB 2.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [4]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


### 토크나이징이 완료된 위키 백과 코퍼스 다운로드 및 불용어 사전 크롤링
[데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)
 
[불용어 사전 출처](https://www.ranks.nl/stopwords/korean)

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [36]:
%cd /content/drive/MyDrive/nlp/w2v 

/content/drive/MyDrive/nlp/w2v


In [38]:
# 데이터 다운로드
!pip install gdown
!gdown https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx
!unzip tokenized.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx
To: /content/drive/MyDrive/nlp/w2v/tokenized.zip
100% 873M/873M [00:06<00:00, 139MB/s]
Archive:  tokenized.zip
   creating: tokenized/
  inflating: tokenized/korquad_mecab.txt  
  inflating: tokenized/wiki_ko_mecab.txt  
  inflating: tokenized/corpus_mecab_jamo.txt  
  inflating: tokenized/ratings_okt.txt  
  inflating: tokenized/ratings_khaiii.txt  
  inflating: tokenized/ratings_hannanum.txt  
  inflating: tokenized/ratings_soynlp.txt  
  inflating: tokenized/ratings_mecab.txt  
  inflating: tokenized/ratings_komoran.txt  


In [11]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [12]:
stop_words[0]

'아'

### 단어 사전 구축 함수 구현 
- 문서 리스트를 입력 받아 사전을 생성하는 함수 
- 함수 정의
    - 입력 매개변수
        - docs : 문서 리스트
        - min_count : 최소 단어 등장 빈도수 (단어 빈도가 `min_count` 미만인 단어는 사전에 포함하지 않음)

    - 반환값 
        - word2count : 단어별 빈도 사전 (key: 단어, value: 등장 횟수)
        - wid2word : 단어별 인덱스(wid) 사전 (key: 단어 인덱스(int), value: 단어)
        - word2wid : 인덱스(wid)별 단어 사전 (key: 단어, value: 단어 인덱스(int))

In [40]:
# 코퍼스 로드
_DATA_DIR="/content/drive/MyDrive/nlp/w2v/tokenized"

with open(os.path.join(_DATA_DIR, "wiki_ko_mecab.txt")) as reader:
    docs = reader.readlines()

In [41]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,237


In [42]:
# 500개로 문서 개수를 줄임
docs=random.sample(docs,500)
len(docs)

500

In [48]:
# 영어, 특수문자 제거 
docs = [re.compile('[ㄱ-ㅣ가-힣]+').findall(str(doc)) for doc in docs]
print(f"Check : {docs[0][:1000]}")

Check : ['남모', '공주', '는', '신라', '의', '공주', '왕족', '으로', '법흥왕', '과', '보과', '공주', '부여', '씨', '의', '딸', '이', '며', '백제', '동성왕', '의', '외손녀', '였', '다', '경쟁자', '인', '준정', '과', '함께', '신라', '의', '초대', '여성', '원화', '화랑', '였', '다', '그', '가', '준정', '에게', '암살', '당한', '것', '을', '계기', '로', '화랑', '은', '여성', '이', '아닌', '남성', '미소년', '으로', '선발', '하', '게', '되', '었', '다', '신라', '진흥왕', '에게', '는', '사촌', '누나', '이', '자', '이모', '가', '된다', '신라', '의', '청소년', '조직', '이', '었', '던', '화랑도', '는', '처음', '에', '는', '남모', '준정', '두', '미녀', '를', '뽑', '아', '이', '를', '원화', '라', '했으며', '이', '들', '주위', '에', '는', '여', '명', '의', '무리', '를', '따르', '게', '하', '였', '다', '그러나', '준정', '과', '남모', '는', '서로', '최고', '가', '되', '고자', '시기', '하', '였', '다', '준정', '은', '박영실', '을', '섬겼', '는데', '지소태후', '는', '자신', '의', '두', '번', '째', '남편', '이', '기', '도', '한', '그', '를', '싫어해서', '준정', '의', '원화', '를', '없애', '고', '낭도', '가', '부족', '한', '남모', '에게', '위화랑', '의', '낭도', '를', '더', '해', '주', '었', '다', '그', '뒤', '남모', '는', '준정', '의', '초대', '로', '그', '의', '집', '에', 

In [50]:
def make_vocab(docs:List[str], min_count:int):
    """
    'docs'문서 리스트를 입력 받아 단어 사전을 생성.
    
    return 
        - word2count : 단어별 빈도 사전
        - wid2word : 단어별 인덱스(wid) 사전 
        - word2wid : 인덱스(wid)별 단어 사전
    """

    word2count = dict()
    word2id = dict()
    id2word = dict()

    _word2count = dict() # 단어별 등장 빈도를 기록하기 위한 임시 딕셔너리 생성
    for doc in tqdm(docs):
        # 단어 개수가 3개 이하는 skip
        if len(doc)>3:
            for word in doc:
                # 불용어 리스트에 포함된 단어 제거
                if word in stop_words:
                    continue
                # 임시 딕셔너리에 단어별 등장 빈도 기록 
                try:
                    _word2count[word]+=1
                except KeyError:
                    _word2count[word]=1

    # 토큰 최소 빈도를 만족하는 토큰만 사전(word2count)에 추가
    idx=0
    for w,c in _word2count.items():
        if c<min_count:
            continue
        word2count[w] = c
        word2id[w] = idx
        id2word[idx] = w
        idx+=1

    
    return word2count, word2id, id2word

In [51]:
word2count, word2id, id2word = make_vocab(docs, min_count=5)

100%|██████████| 500/500 [00:02<00:00, 201.94it/s]


In [52]:
doc_len = sum(word2count.values()) # 문서 내 모든 단어의 개수 (unique한 단어 개수 * 단어 등장 빈도)
print(f"{doc_len:,}")

161,255


In [53]:
print(f"# unique word : {len(word2id):,}")

# unique word : 5,661


### Dataset 클래스 구현
- Skip-Gram 방식의 학습 데이터 셋(`Tuple(target_word, context_word)`)을 생성하는 `CustomDataset` 
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - docs: 문서 리스트
        - word2id: 단어별 인덱스(wid) 사전
        - window_size: Skip-Gram의 윈도우 사이즈
    - 메소드
        - `make_pair()`
            - 문서를 단어로 쪼개고, 사전에 존재하는 단어들만 단어 인덱스로 변경
            - Skip-gram 방식의 `(target_word, context_word)` 페어(tuple)들을 `pairs` 리스트에 담아 반환
        - `__len__()`
            - `pairs` 리스트의 개수 반환
        - `__getitem__(index)`
            - `pairs` 리스트를 인덱싱

In [56]:
class CustomDataset(Dataset):

    def __init__(self, docs:List[str], word2id:Dict[str,int], window_size:int=5):
        self.docs = docs
        self.word2id = word2id
        self.window_size = window_size
        self.pairs = self.make_pair()
    
    def make_pair(self):
        """
        (target, context) 형식의 Skip-gram pair 데이터 셋 생성 
        """
        pairs = []
        for doc in tqdm(self.docs):
            word_ids = [] # 문서 내 (사전에 존재하는) 단어의 인덱스를 저장하기 위한 리스트 
            for word in doc:
                # 사전(word2id)에 존재하는 단어만 단어 인덱스(wid)로 변경해 학습 데이터에 추가
                if self.word2id.get(word):
                    word_ids.append(self.word2id.get(word))
                else:
                    continue

            # 학습 데이터 구축
            for i, u in enumerate(word_ids):
                for j in range(self.window_size):
                    if i-1-j>=0: # target_word 기준 왼쪽 방향의 context_word들을 추가
                        pairs.append((u, word_ids[i-1-j]))
                    if i+1+j<len(word_ids): # target_word 기준 오른쪽 방향의 context_word들을 추가
                        pairs.append((u, word_ids[i+1+j]))
        return pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]

In [57]:
dataset = CustomDataset(docs, word2id, window_size=5)

100%|██████████| 500/500 [00:00<00:00, 696.52it/s]


In [58]:
len(dataset)

1597490

In [62]:
dataset[0]

(1, 2)

In [60]:
# verify (target word, context word)
for i, pair in enumerate(dataset):
    if i==100:
        break
    print(f"({id2word[pair[0]]}, {id2word[pair[1]]})")
    

(공주, 는)
(공주, 신라)
(공주, 공주)
(공주, 왕족)
(공주, 공주)
(는, 공주)
(는, 신라)
(는, 공주)
(는, 왕족)
(는, 공주)
(는, 부여)
(신라, 는)
(신라, 공주)
(신라, 공주)
(신라, 왕족)
(신라, 공주)
(신라, 부여)
(신라, 씨)
(공주, 신라)
(공주, 왕족)
(공주, 는)
(공주, 공주)
(공주, 공주)
(공주, 부여)
(공주, 씨)
(공주, 딸)
(왕족, 공주)
(왕족, 공주)
(왕족, 신라)
(왕족, 부여)
(왕족, 는)
(왕족, 씨)
(왕족, 공주)
(왕족, 딸)
(왕족, 며)
(공주, 왕족)
(공주, 부여)
(공주, 공주)
(공주, 씨)
(공주, 신라)
(공주, 딸)
(공주, 는)
(공주, 며)
(공주, 공주)
(공주, 백제)
(부여, 공주)
(부여, 씨)
(부여, 왕족)
(부여, 딸)
(부여, 공주)
(부여, 며)
(부여, 신라)
(부여, 백제)
(부여, 는)
(부여, 였)
(씨, 부여)
(씨, 딸)
(씨, 공주)
(씨, 며)
(씨, 왕족)
(씨, 백제)
(씨, 공주)
(씨, 였)
(씨, 신라)
(씨, 다)
(딸, 씨)
(딸, 며)
(딸, 부여)
(딸, 백제)
(딸, 공주)
(딸, 였)
(딸, 왕족)
(딸, 다)
(딸, 공주)
(딸, 인)
(며, 딸)
(며, 백제)
(며, 씨)
(며, 였)
(며, 부여)
(며, 다)
(며, 공주)
(며, 인)
(며, 왕족)
(며, 준정)
(백제, 며)
(백제, 였)
(백제, 딸)
(백제, 다)
(백제, 씨)
(백제, 인)
(백제, 부여)
(백제, 준정)
(백제, 공주)
(백제, 신라)
(였, 백제)
(였, 다)
(였, 며)
(였, 인)
(였, 딸)


### 위에서 생성한 `dataset`으로 DataLoader  객체 생성


In [63]:
train_dataloader = DataLoader(
    dataset, batch_size=64, shuffle=True
)

In [64]:
len(train_dataloader)

24961

### Negative Sampling 함수 구현
- Skip-Gram은 복잡도를 줄이기 위한 방법으로 negative sampling을 사용한다. 
[negative distribution 설명](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#How-are-negative-samples-drawn?)

- 함수 정의
    - 입력 매개변수
        - batch_size : 배치 사이즈, matrix의 row 개수 
        - n_neg_sample : negative sample의 개수, matrix의 column 개수
    - 반환값 
        - neg_v : 추출된 negative sample (2차원의 리스트)


In [65]:
# negative sample을 추출할 sample table 생성 
sample_table = []
sample_table_size = doc_len

# noise distribution 생성
alpha = 3/4
frequency_list = np.array(list(word2count.values())) ** alpha
Z = sum(frequency_list)
ratio = frequency_list/Z
negative_sample_dist = np.round(ratio*sample_table_size)

for wid, c in enumerate(negative_sample_dist):
    sample_table.extend([wid]*int(c))

In [67]:
len(sample_table)

161091

In [68]:
def get_neg_v_negative_sampling(batch_size:int, n_neg_sample:int):
    """
    (batch_size, n_neg_sample) shape의 네거티브 샘플 메트릭스 생성
    """
    neg_v = np.random.choice(
        sample_table, size=(batch_size, n_neg_sample)
    ).tolist()
    return neg_v

In [69]:
get_neg_v_negative_sampling(4, 5)

[[36, 4422, 52, 2801, 2018],
 [5559, 307, 4189, 371, 5444],
 [3480, 4774, 1336, 271, 180],
 [57, 3460, 1688, 4855, 1636]]

### 미니 튜토리얼
Skip-Gram 모델의 `forward` 및 `loss` 연산 방식
- Reference

    - [Skip-Gram negative sampling loss function 설명 영문 블로그](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#Derivation-of-Cost-Function-in-Negative-Sampling)
    - [Skip-Gram negative sampling loss function 설명 한글 블로그](https://reniew.github.io/22/)

In [70]:
# hyper parameter example
emb_size = 30000 # vocab size
emb_dimension = 300 # word embedding 차원
n_neg_sample = 5
batch_size = 32

In [71]:
# 1. Embedding Matrix와 Context Matrix를 생성
u_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)
v_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)

In [72]:
# 2. wid(단어 인덱스)를 임의로 생성
pos_u = torch.randint(high = emb_size, size = (batch_size,))
pos_v = torch.randint(high = emb_size, size = (batch_size,))
neg_v = get_neg_v_negative_sampling(batch_size, n_neg_sample)
print(f"Target word idx : {pos_u} Pos context word idx : {pos_v} Neg context word idx : {neg_v}\n")

Target word idx : tensor([24460, 10634,  2864, 23952,  3320, 15187, 19625, 26546, 27339,  3920,
        25847,  6023,  5055,  7070,  6291, 10245, 15926,   641, 20178,  4565,
         4784, 26715, 16955, 28742, 17947, 19774,  8065, 22605,  3061, 28965,
         3056, 17963]) Pos context word idx : tensor([23224,  5636, 23712,  5234,  3991, 17897, 25123, 17938, 19634, 24228,
          693,   799, 25457,  1308, 28935, 25696,  5601, 23878,  8312,  1292,
        21380, 16974,  9318,  9578, 12915, 29271, 26465, 20572,  2362, 25929,
        19754, 29080]) Neg context word idx : [[869, 3542, 900, 92, 629], [1015, 4851, 5519, 4237, 1502], [4712, 625, 1569, 762, 1984], [519, 1872, 3984, 1537, 362], [56, 3899, 2549, 14, 5260], [1033, 549, 683, 4227, 623], [2917, 1330, 1041, 2769, 2579], [705, 3040, 4881, 93, 4960], [3878, 285, 115, 2513, 331], [1093, 2828, 539, 114, 3726], [1486, 753, 1632, 4724, 5037], [872, 1911, 3143, 465, 1338], [3285, 346, 1813, 470, 87], [621, 4215, 2547, 4086, 52], [1646, 

In [73]:
# 3. tensor로 변환
pos_u = Variable(torch.LongTensor(pos_u)).to(device)
pos_v = Variable(torch.LongTensor(pos_v)).to(device)
neg_v = Variable(torch.LongTensor(neg_v)).to(device)

In [74]:
# 4. wid로 각각의 embedding matrix에서 word embedding 값을 가져오기
pos_u = u_embedding(pos_u)
pos_v = v_embedding(pos_v)
neg_v = v_embedding(neg_v)
print(f"shape of pos_u embedding : {pos_u.shape}\n shape of pos_v embedding : {pos_v.shape}\n shape of neg_v embedding : {neg_v.shape}")


shape of pos_u embedding : torch.Size([32, 300])
 shape of pos_v embedding : torch.Size([32, 300])
 shape of neg_v embedding : torch.Size([32, 5, 300])


In [75]:
# 5. dot product 
pos_score = torch.mul(pos_u, pos_v) # 행렬 element-wise 곱 (= row 곱 )
print(pos_score.shape)
pos_score = torch.sum(pos_score, dim=1)
print(f"shape of pos logits : {pos_score.shape}\n")

print(neg_v.shape) # 3d tensor (b,n,m)
print(pos_u.shape)
print(pos_u.unsqueeze(dim=2).shape) # last axis에 1차원을 추가 (b,m,p)
neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)) # batch-matrix-matrix multiplication output (b,n,p)
print(neg_score.shape)
neg_score = neg_score.squeeze() 
print(f"shape of logits : {neg_score.shape}")

torch.Size([32, 300])
shape of pos logits : torch.Size([32])

torch.Size([32, 5, 300])
torch.Size([32, 300])
torch.Size([32, 300, 1])
torch.Size([32, 5, 1])
shape of logits : torch.Size([32, 5])


In [76]:
# 6. loss 구하기
pos_score = F.logsigmoid(pos_score)
neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
print(f"pos logits : {pos_score.sum()}")
print(f"neg logits : {neg_score.sum()}")
loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
print(f"Loss : {loss}")

pos logits : -241.4199676513672
neg logits : -1307.4857177734375
Loss : 1548.9056396484375


### Skip-gram 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `SkipGram` 클래스를 구현
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `device` : 연산 장치 종류
    - 생성자에서 생성해야할 변수 
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `u_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (target_word)
        - `v_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (context_word)
    - 메소드
        - `init_embedding()`
            - 엠베딩 메트릭스 값을 초기화
        - `forward()`
            - 위 튜토리얼과 같이 dot product를 수행한 후 score를 생성
            - loss를 반환 (loss 설명 추가)
        - `save_emedding()`
            - `u_embedding`의 단어 엠베딩 값을 단어 별로 파일에 저장


In [77]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, emb_dimension:int, device:str):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dimension = emb_dimension
        self.u_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.v_embedding = nn.Embedding(vocab_size, emb_dimension, sparse=True).to(device)
        self.init_embedding()
    
    
    def init_embedding(self):
        """
        u_embedding과 v_embedding 메트릭스 값을 초기화
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embedding.weight.data.uniform_(-initrange, initrange)
        self.v_embedding.weight.data.uniform_(-0, 0)
    
    
    def forward(self, pos_u, pos_v, neg_v):
        """
        dot product를 수행한 후 score를 생성
        loss 반환
        """    
        # 각각의 embedding matrix에서 word embedding 값을 가져오기
        pos_u = self.u_embedding(pos_u)
        pos_v = self.v_embedding(pos_v)
        neg_v = self.v_embedding(neg_v)
        
        # dot product
        pos_score = torch.mul(pos_u, pos_v)
        pos_score = torch.sum(pos_score, dim=1)

        # loss 구하기 
        pos_score = F.logsigmoid(pos_score)
        neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
        neg_score = F.logsigmoid(-1 * neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
        
        loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
        return loss
    
    def save_embedding(self, id2word, file_name, use_cuda):
        """
        'file_name' 위치에 word와 word_embedding을 line-by로 저장
        파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
        """
        if use_cuda: # parameter를 gpu 메모리에서 cpu 메모리로 옮김
            embedding = self.u_embedding.weight.cpu().data.numpy()
        else:
            embedding = self.u_embedding.weight.data.numpy()
        
        with open(file_name, 'w') as writer:
            # 파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
            writer.write(f"{len(id2word)} {embedding.shape[-1]}\n")
            
            for wid, word in id2word.items():
                e = embedding[wid]
                e = " ".join([str(e_) for e_ in e])
                writer.write(f"{word} {e}\n")
                

### Skip-Gram 방식의  Word2Vec 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `Word2Vec` 클래스를 구현
- 클래스 정의
    - 생성자(`__init__()`) 입력 매개 변수
        - `input_file` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `device` : 연상 장치 종류
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `min_count` : 사전에 추가될 단어의 최소 등장 빈도
    - 생성자에서 생성해야 할 변수 
        - `docs` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `word2count`, `word2id`, `id2word` : 위에서 구현한 `make_vocab()` 함수의 반환 값
        - `device` : 연산 장치 종류
        - `emb_size` : vocab의 (unique한) 단어 종류 
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `model` : `SkipGram` 클래스의 인스턴스
        - `optimizer` : `SGD` 클래스의 인스턴스
    - 메소드
        - `train()`
            - 입력 매개변수 
                - `train_dataloader`
            - Iteration 횟수만큼 input_file 학습 데이터를 학습한다. 매 epoch마다 for loop 돌면서 batch 단위 학습 데이터를 skip gram 모델에 학습함. 학습이 끝나면 word embedding을 output_file_name 파일에 저장.
- Reference
    - [Optimizer - SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [79]:
class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = output_file_name
        self.word2count, self.word2id, self.id2word = make_vocab(self.docs, min_count=min_count)
        self.device = device
        self.emb_size = len(self.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size =batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        self.model = SkipGram(self.emb_size, self.emb_dimension, self.device)
        self.optimizer = SGD(
            self.model.parameters(),
            lr = self.lr
        )
        # train() 함수에서 만든 임베딩 결과 파일들을 저장할 폴더 생성
        os.makedirs(self.output_file_name, exist_ok=True)
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*self.iteration
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1

                pos_u, pos_v = batch
                # negative data 생성
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                
                # 데이터를 tensor화 & device 설정
                pos_u = Variable(torch.LongTensor(pos_u)).to(self.device)
                pos_v = Variable(torch.LongTensor(pos_v)).to(self.device)
                neg_v = Variable(torch.LongTensor(neg_v)).to(self.device)

                # gradient 초기화
                self.optimizer.zero_grad()
                self.model.zero_grad()

                # forward
                loss = self.model.forward(pos_u, pos_v, neg_v)

                # loss
                loss.backward()
                self.optimizer.step()
                self.scheduler.step()

                batch_loss += loss.item()
                total_loss += loss.item()
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")
                    



In [80]:
# Word2Vec 클래스의 인스턴스 생성
output_file = os.path.join(".", "word2vec_wiki")
w2v = Word2Vec(docs, output_file, device, n_neg_sample=10, iteration=3)

100%|██████████| 500/500 [00:02<00:00, 203.86it/s]


In [81]:
# 학습 데이터 셋 및 데이터 로더 생성
dataset = CustomDataset(w2v.docs, w2v.word2id, w2v.window_size)
train_dataloader = DataLoader(dataset, w2v.batch_size)
len(train_dataloader)

100%|██████████| 500/500 [00:00<00:00, 697.81it/s]


24961

In [82]:
# 학습 
w2v.train(train_dataloader)

*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 24961*****
Step: 500 Loss: 483.4350 lr: 0.0199
Step: 1000 Loss: 381.9070 lr: 0.0197
Step: 1500 Loss: 310.7652 lr: 0.0196
Step: 2000 Loss: 259.5921 lr: 0.0195
Step: 2500 Loss: 251.3580 lr: 0.0193
Step: 3000 Loss: 205.1873 lr: 0.0192
Step: 3500 Loss: 165.6209 lr: 0.0191
Step: 4000 Loss: 219.0585 lr: 0.0189
Step: 4500 Loss: 210.9005 lr: 0.0188
Step: 5000 Loss: 200.9764 lr: 0.0187
Step: 5500 Loss: 192.6000 lr: 0.0185
Step: 6000 Loss: 196.7885 lr: 0.0184
Step: 6500 Loss: 194.8715 lr: 0.0183
Step: 7000 Loss: 209.8101 lr: 0.0181
Step: 7500 Loss: 207.7619 lr: 0.0180
Step: 8000 Loss: 196.4875 lr: 0.0179
Step: 8500 Loss: 191.3960 lr: 0.0177
Step: 9000 Loss: 191.1824 lr: 0.0176
Step: 9500 Loss: 159.0769 lr: 0.0175
Step: 10000 Loss: 114.8233 lr: 0.0173
Step: 10500 Loss: 187.4499 lr: 0.0172
Step: 11000 Loss: 190.4287 lr: 0.0171
Step: 11500 Loss: 197.7998 lr: 0.0169
Step: 12000 Loss: 157.9021 lr: 0.0168
Step: 12500 Loss: 195.7434 lr: 0.0167
Step:

# 유사도 계산

학습을 통해 얻어낸 임베딩으로 유사도 계산해보기

In [84]:
import gensim

In [86]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./word2vec_wiki/w2v_2.txt', binary=False)

In [87]:
word_vectors.most_similar(positive=['대통령'])

[('국회의원', 0.9721125960350037),
 ('당', 0.9688351154327393),
 ('민주', 0.966854453086853),
 ('장관', 0.9648937582969666),
 ('학회', 0.9629717469215393),
 ('국민', 0.96233069896698),
 ('해군', 0.9616174697875977),
 ('영조', 0.9608238935470581),
 ('변호사', 0.9607787132263184),
 ('연합', 0.9583690166473389)]

In [88]:
word_vectors.most_similar(positive=['서울'])

[('현대', 0.9180622100830078),
 ('랑', 0.9111118912696838),
 ('화랑', 0.9018165469169617),
 ('갤러리', 0.901596188545227),
 ('국제', 0.9004174470901489),
 ('부산', 0.8989148139953613),
 ('미술관', 0.8983471393585205),
 ('아트', 0.8968197107315063),
 ('샘터', 0.8910189867019653),
 ('페어', 0.8865504264831543)]

In [90]:
word_vectors.most_similar(positive=['남자'], negative=['여자'])

[('쓰', 0.5390145778656006),
 ('용어', 0.5344403982162476),
 ('못하', 0.5001216530799866),
 ('알려', 0.49944472312927246),
 ('는다', 0.49281448125839233),
 ('기술', 0.4882909953594208),
 ('싶', 0.4847915470600128),
 ('형성', 0.48163795471191406),
 ('소프트웨어', 0.47975271940231323),
 ('못했', 0.4789581000804901)]